# Viewing categorical AIS tracks 

To help understand the AIS data, it can be useful to color-code each location by a category, because the behavior of vessels in a given category might differ from vessels in other categories. Here we'll show how to color code by vessel type using Datashader.

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import colorcet as cc
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import rasterize, datashade, dynspread
hv.extension('bokeh')

Defining categories by collapsing the 100 [AIS Vessel Types](https://api.vtexplorer.com/docs/ref-aistypes.html) down to 22 broader groups of vessel types:

In [ ]:
vessel_types=pd.read_csv("AIS_categories.csv")
vessel_types.head(40).tail()

In [ ]:
def vessel_category(val):
    i = int(val)
    cat = int(vessel_types.iloc[i].category) if i in vessel_types.index else 0
    return cat if cat in [0, 2,3,20,12,19] else 22 # limit to most common types

def category_desc(val):
    return vessel_types[vessel_types.category==val].iloc[0].category_desc

In [ ]:
groups = {vessel_category(i):category_desc(vessel_category(i)) for i in vessel_types.num.unique()}
groups

## Loading AIS pings and Vessel information

In [ ]:
%%time
broadcast = pd.read_csv('../Broadcast.csv', parse_dates=[1]) # Zone 10
vessel_info =  pd.read_csv('../Vessel.csv')
vessel_info['vessel_type']= vessel_info['vessel_type'].fillna(0).astype(int) # Mapping NaN to 'Not Available' i.e 0
broadcast.head()

## Assigning broader categories

In [ ]:
# Dict for mmsi:vessel type
vessel_mapping = {k:v for k,v in zip(vessel_info['mmsi_id'], 
                                     vessel_info['vessel_type'].apply(vessel_category))} 
# Join between broadcast AIS pings and vessel info
categories = broadcast['mmsi_id'].apply(lambda x: vessel_mapping.get(x, 0))
broadcast['category'] = categories

### Defining color key and legend

In [ ]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

color_key = {list(groups.keys())[ind]:tuple(int(el*255.) for el in val) for ind,val in 
             enumerate(cc.glasbey_bw_minc_20_hue_330_100[:(len(groups))])} # 23 Glasby colors

In [ ]:
color_names = {groups[k]:rgb_to_hex(v) for k,v in color_key.items()}
color_points = hv.NdOverlay({k: hv.Points([0,0], label=str(k)).opts(color=v, size=0) for k, v in color_names.items()})

In [ ]:
%%time
broadcast.loc[:, 'x'], broadcast.loc[:, 'y'] = \
    hv.util.transform.lon_lat_to_easting_northing(broadcast.lon,broadcast.lat)

# Datashaded, categorical AIS plot (Zone 10)

In [ ]:
tiles  = hv.element.tiles.ESRI().redim(x='easting', y='northing')
points = dynspread(datashade(hv.Points(broadcast, ['x', 'y'], ['category']), 
                             color_key=color_key, aggregator=ds.count_cat('category')))
tiles.opts( width=900, height=500) * color_points * points